DEAP para Rosenbrock funcional

In [ ]:
import random
import time
import csv
import os
import socket
import numpy as np
from codecarbon import EmissionsTracker
from deap import base, creator, tools, algorithms

# ------------------------------------------------------------
# Configuración del problema
# ------------------------------------------------------------
INDIVIDUAL_SIZE = 1024
MAX_GENERATIONS = 1000000
MAX_TIME_SECONDS = 120
RUNS_PER_CONFIG = 10

POPULATION_SIZES = [2**6, 2**10, 2**14]
CROSSOVER_RATES  = [0.01, 0.2, 0.8]

# --- Tasas de mutación estilo DEAP --------------------------
MUTATION_RATE_INDIVIDUAL = 0.1   # p_ind  (se sortea 1 vez por individuo)
MUTATION_RATE_BIT        = 0.1   # p_bit  (se sortea por gen si el indiv. fue elegido)
# -------------------------------------------------------------

# --- Parámetros para Rosenbrock ----------------------------
LOWER_BOUND = -5
UPPER_BOUND = 10
# El mínimo global conocido de Rosenbrock es 0 cuando todos xi=1
KNOWN_OPTIMUM = 0.0
# Valor de referencia para el peor caso (estimación)
WORST_CASE_VALUE = 1e10
# -------------------------------------------------------------

COMPUTER_NAME   = socket.gethostname()
results_file    = f'resultados_rosenbrock_deap_{COMPUTER_NAME}.csv'
codecarbon_file = f'codecarbon_rosenbrock_deap_{COMPUTER_NAME}.csv'

# ------------------------------------------------------------
# Clases y funciones auxiliares
# ------------------------------------------------------------
class EvolutionStats:
    def __init__(self):
        self.initial_fitness   = 0
        self.best_fitness      = 0
        self.gen_best_fitness  = 0
        self.current_generation = 0
        self.termination_cause = "timeout"
        # Almacenar el mejor valor real de Rosenbrock para conversión
        self.best_raw_value = float('inf')
        # Almacenar el peor valor visto para normalización dinámica
        self.worst_raw_value = 0

# Instancia global para almacenar estadísticas
stats = EvolutionStats()

# Función de fitness (Rosenbrock sin normalizar - DEAP se encarga de minimización/maximización)
def rosenbrock_function(individual):
    global stats
    
    # Calcular valor Rosenbrock (a minimizar)
    raw_value = 0
    for i in range(len(individual) - 1):
        raw_value += 100 * (individual[i+1] - individual[i]**2)**2 + (individual[i] - 1)**2
    
    # Limitar valores extremos para evitar problemas numéricos
    if raw_value > WORST_CASE_VALUE:
        raw_value = WORST_CASE_VALUE
        
    # Actualizar el peor valor visto (para normalización dinámica)
    if raw_value > stats.worst_raw_value:
        stats.worst_raw_value = raw_value
        
    # Actualizar mejor valor bruto si corresponde
    if raw_value < stats.best_raw_value:
        stats.best_raw_value = raw_value
        
    # Normalización invertida: 0 = peor (WORST_CASE_VALUE), 1 = mejor (0)
    # Usamos la peor estimación vista hasta el momento para la normalización
    normalized_fitness = 1.0 - (raw_value / (stats.worst_raw_value + 1e-10))
    
    # Asegurar que el fitness esté en el rango [0,1]
    normalized_fitness = max(0.0, min(1.0, normalized_fitness))
    
    return normalized_fitness,  # DEAP requiere una tupla para el fitness

# Mutación personalizada al estilo DEAP para valores reales
def custom_mutation(individual, indpb, bitpb, sigma):
    """
    Versión personalizada de la mutación para valores reales:
    1) indpb = probabilidad de mutar cada individuo.
    2) Si se muta, bitpb = probabilidad de mutar cada gen.
    3) Para cada gen seleccionado, se aplica una perturbación gaussiana.
    """
    # Decidir si el individuo debe ser mutado
    if random.random() < indpb:
        # Mutar los genes seleccionados con probabilidad bitpb
        for i in range(len(individual)):
            if random.random() < bitpb:
                # Mutación gaussiana
                individual[i] += random.gauss(0, sigma)
                # Asegurar que se mantiene dentro de límites
                individual[i] = max(min(individual[i], UPPER_BOUND), LOWER_BOUND)
                
    return individual,

# SBX Crossover personalizado con manejo de errores
def safe_sbx_crossover(ind1, ind2, eta, prob):
    """
    Implementación personalizada de SBX (Simulated Binary Crossover)
    con protección contra desbordamiento numérico.
    """
    if random.random() > prob:
        return ind1, ind2
    
    size = min(len(ind1), len(ind2))
    
    for i in range(size):
        # Evitar la división por cero
        if abs(ind1[i] - ind2[i]) > 1e-10:
            # Ordenar los valores
            if ind1[i] < ind2[i]:
                y1, y2 = ind1[i], ind2[i]
                swap = False
            else:
                y1, y2 = ind2[i], ind1[i]
                swap = True
                
            # Cálculos SBX con protección contra desbordamiento
            try:
                # Limitar la magnitud del exponente para evitar desbordamiento
                beta = 1.0 + (2.0 * (y1 - LOWER_BOUND) / (y2 - y1))
                alpha = 2.0 - min(100.0, beta**(eta + 1.0))
                
                u = random.random()
                if u <= 1.0 / alpha:
                    beta_q = (u * alpha) ** (1.0 / (eta + 1.0))
                else:
                    beta_q = (1.0 / (2.0 - u * alpha)) ** (1.0 / (eta + 1.0))
                
                c1 = 0.5 * ((y1 + y2) - beta_q * (y2 - y1))
                c2 = 0.5 * ((y1 + y2) + beta_q * (y2 - y1))
                
                # Mantener dentro de límites
                c1 = max(LOWER_BOUND, min(UPPER_BOUND, c1))
                c2 = max(LOWER_BOUND, min(UPPER_BOUND, c2))
                
                # Asignar de vuelta a los individuos
                if not swap:
                    ind1[i], ind2[i] = c1, c2
                else:
                    ind1[i], ind2[i] = c2, c1
                    
            except (OverflowError, ValueError, ZeroDivisionError):
                # En caso de error, aplicar cruce aritmético simple
                blend = random.random()
                ind1[i] = blend * ind1[i] + (1 - blend) * ind2[i]
                ind2[i] = (1 - blend) * ind1[i] + blend * ind2[i]
    
    return ind1, ind2

# Callback para checkpoints y estadísticas
def stats_callback(gen, population, halloffame, stats_obj=None):
    global stats
    
    if not population:
        return False
    
    current_best = max(ind.fitness.values[0] for ind in population)
    
    # Para primeras generaciones o cada 50, mostrar información
    if gen < 3 or gen % 50 == 0:
        print(f"  Gen {gen}: fitness={current_best:.6f}, "
              f"raw={stats.best_raw_value:.2e}, "
              f"worst_seen={stats.worst_raw_value:.2e}")
    
    if gen == 0:
        stats.initial_fitness = current_best
    if current_best > stats.best_fitness:
        stats.best_fitness = current_best
        stats.gen_best_fitness = gen
    
    # Actualizar generación actual
    stats.current_generation = gen
    
    # Comprobar condición de tiempo para terminar
    elapsed_time = time.time() - stats.start_time
    if elapsed_time >= MAX_TIME_SECONDS:
        stats.termination_cause = "timeout"
        return True
    
    return False

# ------------------------------------------------------------
# Ejecución principal
# ------------------------------------------------------------
def main():
    # Crear archivo CSV de resultados si no existe
    file_exists = os.path.isfile(results_file)
    with open(results_file, 'a', newline='') as csvfile:
        fieldnames = ['population_size', 'crossover_rate', 'mutation_individual_rate',
                      'mutation_bit_rate', 'run', 'generations', 'initial_fitness',
                      'best_fitness', 'fitness_variation', 'time', 'energy_consumed',
                      'rosenbrock_value', 'worst_rosenbrock_seen']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()

        # Bucle sobre combinaciones de parámetros
        for pop_size in POPULATION_SIZES:
            for cx_rate in CROSSOVER_RATES:
                for run in range(1, RUNS_PER_CONFIG + 1):
                    print(f"Ejecutando: Población={pop_size}, Cruce={cx_rate}, "
                          f"Run={run}")
                    
                    # Reiniciar estadísticas
                    global stats
                    stats = EvolutionStats()
                    
                    # Configurar semilla aleatoria
                    seed = int(time.time() + run)
                    random.seed(seed)
                    np.random.seed(seed)
                    
                    # Registrar tiempo de inicio
                    stats.start_time = time.time()
                    
                    # Registro y tracking de emisiones con CodeCarbon
                    tracker_id = f"rosenbrock_p{pop_size}_cx{cx_rate}_r{run}"
                    output_dir = os.path.dirname(os.path.abspath(codecarbon_file)) or '.'
                    output_file = os.path.basename(codecarbon_file)
                    
                    # Asegurarnos de que el directorio existe
                    if not os.path.exists(output_dir):
                        try:
                            os.makedirs(output_dir)
                            print(f"Creado directorio: {output_dir}")
                        except Exception as e:
                            print(f"Error al crear directorio: {e}")
                    
                    # Configurar variables de entorno
                    os.environ["CODECARBON_RUN_ID"] = tracker_id
                    
                    try:
                        tracker = EmissionsTracker(
                            project_name="Rosenbrock_Experiment",
                            output_dir=output_dir,
                            output_file=output_file,
                            save_to_file=True,
                            log_level="critical",
                            tracking_mode="process",
                            measure_power_secs=15,
                        )
                        print(f"Tracker inicializado: ID={tracker_id}")
                        
                        tracker.start()
                        tracker_started = True
                        print(f"Emisiones: tracking iniciado")
                    except Exception as e:
                        print(f"Error con CodeCarbon: {e}")
                        tracker_started = False
                    
                    # Configuración de DEAP
                    # Registrar tipos de fitness (maximización)
                    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
                    creator.create("Individual", list, fitness=creator.FitnessMax)
                    
                    toolbox = base.Toolbox()
                    
                    # Registro de operadores
                    # Generador de individuos - inicializa alrededor del centro del rango
                    def init_individual():
                        mean = (LOWER_BOUND + UPPER_BOUND) / 2.0
                        spread = (UPPER_BOUND - LOWER_BOUND) / 4.0
                        return [random.uniform(mean - spread, mean + spread) 
                                for _ in range(INDIVIDUAL_SIZE)]
                    
                    toolbox.register("individual", tools.initIterate, creator.Individual, init_individual)
                    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
                    
                    # Operadores genéticos
                    toolbox.register("evaluate", rosenbrock_function)
                    toolbox.register("select", tools.selTournament, tournsize=2)
                    
                    # Factor de escala para la mutación gaussiana
                    sigma = (UPPER_BOUND - LOWER_BOUND) * 0.1
                    
                    # Registrar operadores personalizados
                    toolbox.register("mate", safe_sbx_crossover, eta=2.0, prob=cx_rate)
                    toolbox.register("mutate", custom_mutation, 
                                    indpb=MUTATION_RATE_INDIVIDUAL, 
                                    bitpb=MUTATION_RATE_BIT, 
                                    sigma=sigma)
                    
                    # Crear población inicial
                    population = toolbox.population(n=pop_size)
                    
                    # Hall of Fame para conservar mejores soluciones
                    hof = tools.HallOfFame(1)
                    
                    # Ejecutar algoritmo evolutivo
                    try:
                        # Inicialización - evaluar población inicial
                        invalid_ind = [ind for ind in population if not ind.fitness.valid]
                        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
                        for ind, fit in zip(invalid_ind, fitnesses):
                            ind.fitness.values = fit
                        
                        # Evolución con callback manual para controlar terminación
                        gen = 0
                        stop_evolution = False
                        
                        while gen < MAX_GENERATIONS and not stop_evolution:
                            # Seleccionar la siguiente generación
                            offspring = toolbox.select(population, len(population))
                            
                            # Clonar los individuos seleccionados
                            offspring = list(map(toolbox.clone, offspring))
                            
                            # Aplicar crossover
                            for i in range(1, len(offspring), 2):
                                if i < len(offspring):  # Asegurar que hay un par
                                    toolbox.mate(offspring[i-1], offspring[i])
                                    del offspring[i-1].fitness.values
                                    del offspring[i].fitness.values
                            
                            # Aplicar mutación a todos los individuos
                            for i in range(len(offspring)):
                                toolbox.mutate(offspring[i])
                                if not offspring[i].fitness.valid:
                                    del offspring[i].fitness.values
                            
                            # Evaluar individuos con fitness inválido
                            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
                            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
                            for ind, fit in zip(invalid_ind, fitnesses):
                                ind.fitness.values = fit
                            
                            # Reemplazo generacional
                            population[:] = offspring
                            
                            # Actualizar Hall of Fame
                            hof.update(population)
                            
                            # Comprobar terminación con callback
                            stop_evolution = stats_callback(gen, population, hof)
                            
                            gen += 1
                            
                    except Exception as e:
                        print(f"Error durante la evolución: {e}")
                    
                    # Tiempo y emisiones
                    elapsed_time = time.time() - stats.start_time
                    
                    # Guardar emisiones
                    if tracker_started:
                        try:
                            emissions = tracker.stop()
                            print(f"Emisiones registradas: {emissions or 0:.6f} kWh")
                        except Exception as e:
                            print(f"Error al detener tracker: {e}")
                            emissions = 0
                    else:
                        emissions = 0
                    
                    # Verificar si el archivo de emisiones existe después de la ejecución
                    emissions_file_path = os.path.join(output_dir, output_file)
                    if os.path.exists(emissions_file_path):
                        print(f"Archivo de emisiones creado correctamente")
                    else:
                        print(f"ADVERTENCIA: No se creó el archivo de emisiones")
                        # Intentar guardar manualmente las emisiones
                        try:
                            with open(emissions_file_path, 'a', newline='') as f:
                                f.write(f"manual_entry,{tracker_id},{emissions or 0},{time.time()}\n")
                            print(f"Creado manualmente un registro básico de emisiones")
                        except Exception as e:
                            print(f"Error al crear registro manual: {e}")
                    
                    # Estadísticas finales
                    generations = stats.current_generation + 1
                    initial_fitness = stats.initial_fitness
                    best_fitness = stats.best_fitness
                    fitness_variation = best_fitness - initial_fitness
                    
                    # Registro en CSV
                    writer.writerow({
                        'population_size'        : pop_size,
                        'crossover_rate'         : cx_rate,
                        'mutation_individual_rate': MUTATION_RATE_INDIVIDUAL,
                        'mutation_bit_rate'      : MUTATION_RATE_BIT,
                        'run'                    : run,
                        'generations'            : generations,
                        'initial_fitness'        : initial_fitness,
                        'best_fitness'           : best_fitness,
                        'fitness_variation'      : fitness_variation,
                        'time'                   : elapsed_time,
                        'energy_consumed'        : emissions,
                        'rosenbrock_value'       : stats.best_raw_value,
                        'worst_rosenbrock_seen'  : stats.worst_raw_value
                    })
                    csvfile.flush()
                    
                    print(f"Generaciones: {generations} | "
                          f"Aptitud inicial={initial_fitness:.6f} → "
                          f"mejor={best_fitness:.6f} | "
                          f"Δ={fitness_variation:.6f}")
                    print(f"Valor Rosenbrock: mejor={stats.best_raw_value:.4e}, "
                          f"peor visto={stats.worst_raw_value:.4e}")
                    print(f"Tiempo: {elapsed_time:.2f}s | "
                          f"Energía: {emissions:.4f} kWh | "
                          f"Parada: {stats.termination_cause}")
                    print("-" * 60)
                    
                    # Limpiar las definiciones creadas por DEAP para evitar duplicados
                    if hasattr(creator, "FitnessMax"):
                        delattr(creator, "FitnessMax")
                    if hasattr(creator, "Individual"):
                        delattr(creator, "Individual")

if __name__ == "__main__":
    main()

DEAP para Schwefel funcional

In [ ]:
import random
import time
import csv
import os
import socket
import numpy as np
from codecarbon import EmissionsTracker
from deap import base, creator, tools, algorithms

# ------------------------------------------------------------
# Configuración del problema
# ------------------------------------------------------------
INDIVIDUAL_SIZE = 1024
MAX_GENERATIONS = 1000000
MAX_TIME_SECONDS = 120
RUNS_PER_CONFIG = 10

POPULATION_SIZES = [2**6, 2**10, 2**14]
CROSSOVER_RATES  = [0.01, 0.2, 0.8]

# ---------------- Tasas de mutación --------------------------
MUTATION_RATE_INDIVIDUAL = 0.1   # p_ind  (se sortea 1 vez por individuo)
MUTATION_RATE_BIT        = 0.1   # p_bit  (se sortea por gen si el indiv. fue elegido)
# -------------------------------------------------------------

# --- Parámetros para Schwefel ----------------------------
LOWER_BOUND = -500  # Límite típico para Schwefel
UPPER_BOUND = 500   # Límite típico para Schwefel
# El mínimo global conocido de Schwefel es en 420.9687 para cada xi
KNOWN_OPTIMUM = 0.0
# Valor de referencia para el peor caso (estimación)
WORST_CASE_VALUE = INDIVIDUAL_SIZE * (418.9829 + UPPER_BOUND)
# -------------------------------------------------------------

COMPUTER_NAME   = socket.gethostname()
results_file    = f'resultados_schwefel_deap_{COMPUTER_NAME}.csv'
codecarbon_file = f'codecarbon_schwefel_deap_{COMPUTER_NAME}.csv'

# ------------------------------------------------------------
# Clases y funciones auxiliares
# ------------------------------------------------------------
class EvolutionStats:
    def __init__(self):
        self.initial_fitness   = 0
        self.best_fitness      = 0
        self.gen_best_fitness  = 0
        self.current_generation = 0
        self.termination_cause = "timeout"
        # Almacenar el mejor valor real para conversión
        self.best_raw_value = float('inf')
        # Almacenar el peor valor visto para normalización dinámica
        self.worst_raw_value = 0
        # Almacenar tiempo de inicio para control de terminación
        self.start_time = 0

# Instancia global para almacenar estadísticas
stats = EvolutionStats()

# Función de fitness (Schwefel sin normalizar - DEAP se encarga de minimización/maximización)
def schwefel_function(individual):
    global stats
    
    # Calcular valor Schwefel (a minimizar)
    raw_value = 0
    D = len(individual)
    
    for i in range(D):
        # La fórmula estándar de Schwefel
        raw_value += -individual[i] * np.sin(np.sqrt(abs(individual[i])))
    
    # Factor de escala para normalizar
    raw_value = 418.9829 * D - raw_value
    
    # Limitar valores extremos para evitar problemas numéricos
    if raw_value > WORST_CASE_VALUE:
        raw_value = WORST_CASE_VALUE
        
    # Actualizar el peor valor visto (para normalización dinámica)
    if raw_value > stats.worst_raw_value:
        stats.worst_raw_value = raw_value
        
    # Actualizar mejor valor bruto si corresponde
    if raw_value < stats.best_raw_value:
        stats.best_raw_value = raw_value
    
    # Estimación del valor máximo posible (peor caso teórico)
    theoretical_worst = 418.9829 * D + D * UPPER_BOUND
    
    # Usar el peor valor visto hasta ahora o el teórico, el que sea mayor
    normalization_factor = max(stats.worst_raw_value, theoretical_worst)
    
    # Normalización invertida: igual que en ParadisEO
    normalized_fitness = 1.0 - (raw_value / (normalization_factor + 1e-10))
    
    # Asegurar que el fitness esté en el rango [0,1]
    normalized_fitness = max(0.0, min(1.0, normalized_fitness))
    
    return normalized_fitness,  # tupla para el fitness

# Mutación personalizada para valores reales
def custom_mutation(individual, indpb, bitpb, sigma):
    """
    Versión personalizada de la mutación para valores reales:
    1) indpb = probabilidad de mutar cada individuo.
    2) Si se muta, bitpb = probabilidad de mutar cada gen.
    3) Para cada gen seleccionado, se aplica una perturbación gaussiana.
    """
    # Decidir si el individuo debe ser mutado
    if random.random() < indpb:
        # Mutar los genes seleccionados con probabilidad bitpb
        for i in range(len(individual)):
            if random.random() < bitpb:
                # Mutación gaussiana
                individual[i] += random.gauss(0, sigma)
                # Asegurar que se mantiene dentro de límites
                individual[i] = max(min(individual[i], UPPER_BOUND), LOWER_BOUND)
                
    return individual,

# SBX Crossover personalizado con manejo de errores
def safe_sbx_crossover(ind1, ind2, eta, prob):
    """
    Implementación personalizada de SBX (Simulated Binary Crossover)
    con protección contra desbordamiento numérico.
    """
    if random.random() > prob:
        return ind1, ind2
    
    size = min(len(ind1), len(ind2))
    
    for i in range(size):
        # Evitar la división por cero
        if abs(ind1[i] - ind2[i]) > 1e-10:
            # Ordenar los valores
            if ind1[i] < ind2[i]:
                y1, y2 = ind1[i], ind2[i]
                swap = False
            else:
                y1, y2 = ind2[i], ind1[i]
                swap = True
                
            # Cálculos SBX con protección contra desbordamiento
            try:
                # Limitar la magnitud del exponente para evitar desbordamiento
                beta = 1.0 + (2.0 * (y1 - LOWER_BOUND) / (y2 - y1))
                alpha = 2.0 - min(100.0, beta**(eta + 1.0))
                
                u = random.random()
                if u <= 1.0 / alpha:
                    beta_q = (u * alpha) ** (1.0 / (eta + 1.0))
                else:
                    beta_q = (1.0 / (2.0 - u * alpha)) ** (1.0 / (eta + 1.0))
                
                c1 = 0.5 * ((y1 + y2) - beta_q * (y2 - y1))
                c2 = 0.5 * ((y1 + y2) + beta_q * (y2 - y1))
                
                # Mantener dentro de límites
                c1 = max(LOWER_BOUND, min(UPPER_BOUND, c1))
                c2 = max(LOWER_BOUND, min(UPPER_BOUND, c2))
                
                # Asignar de vuelta a los individuos
                if not swap:
                    ind1[i], ind2[i] = c1, c2
                else:
                    ind1[i], ind2[i] = c2, c1
                    
            except (OverflowError, ValueError, ZeroDivisionError):
                # En caso de error, aplicar cruce aritmético simple
                blend = random.random()
                ind1[i] = blend * ind1[i] + (1 - blend) * ind2[i]
                ind2[i] = (1 - blend) * ind1[i] + blend * ind2[i]
    
    return ind1, ind2

# Callback para checkpoints y estadísticas
def stats_callback(gen, population, halloffame, stats_obj=None):
    global stats
    
    if not population:
        return False
    
    current_best = max(ind.fitness.values[0] for ind in population)
    
    # Para primeras generaciones o cada 50, mostrar información
    if gen < 3 or gen % 50 == 0:
        print(f"  Gen {gen}: fitness={current_best:.6f}, "
              f"raw={stats.best_raw_value:.2e}, "
              f"worst_seen={stats.worst_raw_value:.2e}")
    
    if gen == 0:
        stats.initial_fitness = current_best
    if current_best > stats.best_fitness:
        stats.best_fitness = current_best
        stats.gen_best_fitness = gen
    
    # Actualizar generación actual
    stats.current_generation = gen
    
    # Comprobar condición de tiempo para terminar
    elapsed_time = time.time() - stats.start_time
    if elapsed_time >= MAX_TIME_SECONDS:
        stats.termination_cause = "timeout"
        return True
    
    # Comprobar condición de generaciones para terminar
    if gen >= MAX_GENERATIONS:
        stats.termination_cause = "max_generations"
        return True
        
    return False

# ------------------------------------------------------------
# Ejecución principal
# ------------------------------------------------------------
def main():
    # Crear archivo CSV de resultados si no existe
    file_exists = os.path.isfile(results_file)
    with open(results_file, 'a', newline='') as csvfile:
        fieldnames = ['population_size', 'crossover_rate', 'mutation_individual_rate',
                      'mutation_bit_rate', 'run', 'generations', 'initial_fitness',
                      'best_fitness', 'fitness_variation', 'time', 'energy_consumed',
                      'schwefel_value', 'worst_schwefel_seen']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()

        # Bucle sobre combinaciones de parámetros
        for pop_size in POPULATION_SIZES:
            for cx_rate in CROSSOVER_RATES:
                for run in range(1, RUNS_PER_CONFIG + 1):
                    print(f"Ejecutando: Población={pop_size}, Cruce={cx_rate}, "
                          f"Run={run}")
                    
                    # Reiniciar estadísticas
                    global stats
                    stats = EvolutionStats()
                    
                    # Configurar semilla aleatoria
                    seed = int(time.time() + run)
                    random.seed(seed)
                    np.random.seed(seed)
                    
                    # Registrar tiempo de inicio
                    stats.start_time = time.time()
                    
                    # Registro y tracking de emisiones con CodeCarbon
                    tracker_id = f"schwefel_p{pop_size}_cx{cx_rate}_r{run}"
                    output_dir = os.path.dirname(os.path.abspath(codecarbon_file)) or '.'
                    output_file = os.path.basename(codecarbon_file)
                    
                    # Asegurarnos de que el directorio existe
                    if not os.path.exists(output_dir):
                        try:
                            os.makedirs(output_dir)
                            print(f"Creado directorio: {output_dir}")
                        except Exception as e:
                            print(f"Error al crear directorio: {e}")
                    
                    # Configurar variables de entorno
                    os.environ["CODECARBON_RUN_ID"] = tracker_id
                    
                    try:
                        tracker = EmissionsTracker(
                            project_name="Schwefel_Experiment",
                            output_dir=output_dir,
                            output_file=output_file,
                            save_to_file=True,
                            log_level="critical",
                            tracking_mode="process",
                            measure_power_secs=15,
                        )
                        print(f"Tracker inicializado: ID={tracker_id}")
                        
                        tracker.start()
                        tracker_started = True
                        print(f"Emisiones: tracking iniciado")
                    except Exception as e:
                        print(f"Error con CodeCarbon: {e}")
                        tracker_started = False
                    
                    # Registrar tipos de fitness (maximización)
                    try:
                        if hasattr(creator, "FitnessMax"):
                            delattr(creator, "FitnessMax")
                        if hasattr(creator, "Individual"):
                            delattr(creator, "Individual")
                            
                        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
                        creator.create("Individual", list, fitness=creator.FitnessMax)
                    except Exception as e:
                        print(f"Error al configurar DEAP: {e}")
                        # Si hay error, intentar usar definiciones existentes
                        pass
                    
                    toolbox = base.Toolbox()
                    
                    # Registro de operadores
                    def init_individual():
                        # Distribución completamente uniforme en el rango [-500, 500]
                        return [random.uniform(LOWER_BOUND, UPPER_BOUND) 
                                for _ in range(INDIVIDUAL_SIZE)]
                    
                    toolbox.register("individual", tools.initIterate, creator.Individual, init_individual)
                    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
                    
                    # Operadores genéticos
                    toolbox.register("evaluate", schwefel_function)
                    toolbox.register("select", tools.selTournament, tournsize=2)
                    
                    # Factor de escala para la mutación gaussiana
                    sigma = (UPPER_BOUND - LOWER_BOUND) * 0.1
                    
                    # Registrar operadores personalizados
                    toolbox.register("mate", safe_sbx_crossover, eta=2.0, prob=cx_rate)
                    toolbox.register("mutate", custom_mutation, 
                                    indpb=MUTATION_RATE_INDIVIDUAL, 
                                    bitpb=MUTATION_RATE_BIT, 
                                    sigma=sigma)
                    
                    # Crear población inicial
                    population = toolbox.population(n=pop_size)
                    
                    # Hall of Fame para conservar mejores soluciones
                    hof = tools.HallOfFame(1)
                    
                    # Ejecutar algoritmo evolutivo
                    try:
                        # Inicialización - evaluar población inicial
                        invalid_ind = [ind for ind in population if not ind.fitness.valid]
                        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
                        for ind, fit in zip(invalid_ind, fitnesses):
                            ind.fitness.values = fit
                        
                        # Evolución con callback manual para controlar terminación
                        gen = 0
                        stop_evolution = False
                        
                        while gen < MAX_GENERATIONS and not stop_evolution:
                            # Seleccionar la siguiente generación
                            offspring = toolbox.select(population, len(population))
                            
                            # Clonar los individuos seleccionados
                            offspring = list(map(toolbox.clone, offspring))
                            
                            # Aplicar crossover
                            for i in range(1, len(offspring), 2):
                                if i < len(offspring):  # Asegurar que hay un par
                                    toolbox.mate(offspring[i-1], offspring[i])
                                    del offspring[i-1].fitness.values
                                    del offspring[i].fitness.values
                            
                            # Aplicar mutación a todos los individuos
                            for i in range(len(offspring)):
                                toolbox.mutate(offspring[i])
                                if not offspring[i].fitness.valid:
                                    del offspring[i].fitness.values
                            
                            # Evaluar individuos con fitness inválido
                            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
                            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
                            for ind, fit in zip(invalid_ind, fitnesses):
                                ind.fitness.values = fit
                            
                            # Reemplazo generacional
                            population[:] = offspring
                            
                            # Actualizar Hall of Fame
                            hof.update(population)
                            
                            # Comprobar terminación con callback
                            stop_evolution = stats_callback(gen, population, hof)
                            
                            gen += 1
                            
                    except Exception as e:
                        print(f"Error durante la evolución: {e}")
                    
                    # Tiempo y emisiones
                    elapsed_time = time.time() - stats.start_time
                    
                    # Guardar emisiones
                    if tracker_started:
                        try:
                            emissions = tracker.stop()
                            print(f"Emisiones registradas: {emissions or 0:.6f} kWh")
                        except Exception as e:
                            print(f"Error al detener tracker: {e}")
                            emissions = 0
                    else:
                        emissions = 0
                    
                    # Verificar si el archivo de emisiones existe después de la ejecución
                    emissions_file_path = os.path.join(output_dir, output_file)
                    if os.path.exists(emissions_file_path):
                        print(f"Archivo de emisiones creado correctamente")
                    else:
                        print(f"ADVERTENCIA: No se creó el archivo de emisiones")
                        # Intentar guardar manualmente las emisiones
                        try:
                            with open(emissions_file_path, 'a', newline='') as f:
                                f.write(f"manual_entry,{tracker_id},{emissions or 0},{time.time()}\n")
                            print(f"Creado manualmente un registro básico de emisiones")
                        except Exception as e:
                            print(f"Error al crear registro manual: {e}")
                    
                    # Estadísticas finales
                    generations = stats.current_generation + 1
                    initial_fitness = stats.initial_fitness
                    best_fitness = stats.best_fitness
                    fitness_variation = best_fitness - initial_fitness
                    
                    # Registro en CSV
                    writer.writerow({
                        'population_size'        : pop_size,
                        'crossover_rate'         : cx_rate,
                        'mutation_individual_rate': MUTATION_RATE_INDIVIDUAL,
                        'mutation_bit_rate'      : MUTATION_RATE_BIT,
                        'run'                    : run,
                        'generations'            : generations,
                        'initial_fitness'        : initial_fitness,
                        'best_fitness'           : best_fitness,
                        'fitness_variation'      : fitness_variation,
                        'time'                   : elapsed_time,
                        'energy_consumed'        : emissions,
                        'schwefel_value'         : stats.best_raw_value,
                        'worst_schwefel_seen'    : stats.worst_raw_value
                    })
                    csvfile.flush()
                    
                    print(f"Generaciones: {generations} | "
                          f"Aptitud inicial={initial_fitness:.6f} → "
                          f"mejor={best_fitness:.6f} | "
                          f"Δ={fitness_variation:.6f}")
                    print(f"Valor Schwefel: mejor={stats.best_raw_value:.4e}, "
                          f"peor visto={stats.worst_raw_value:.4e}")
                    print(f"Tiempo: {elapsed_time:.2f}s | "
                          f"Energía: {emissions:.4f} kWh | "
                          f"Parada: {stats.termination_cause}")
                    print("-" * 60)
                    
                    # Limpiar las definiciones creadas por DEAP para evitar duplicados
                    if hasattr(creator, "FitnessMax"):
                        delattr(creator, "FitnessMax")
                    if hasattr(creator, "Individual"):
                        delattr(creator, "Individual")

if __name__ == "__main__":
    main()